In [ ]:
from google.colab import drive
drive.mount('/content/drive')

##LLava Initialization

In [ ]:
%cd /content
!git clone -b v1.0 https://github.com/camenduru/LLaVA
%cd /content/LLaVA
!pip install -q gradio .

from transformers import AutoTokenizer, BitsAndBytesConfig
from llava.model import LlavaLlamaForCausalLM
import torch

model_path = "4bit/llava-v1.5-13b-3GB"
kwargs = {"device_map": "auto"}
kwargs['load_in_4bit'] = True
kwargs['quantization_config'] = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type='nf4'
)
model = LlavaLlamaForCausalLM.from_pretrained(model_path, low_cpu_mem_usage=True, **kwargs)
tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=False)

vision_tower = model.get_vision_tower()
if not vision_tower.is_loaded:
    vision_tower.load_model()
vision_tower.to(device='cuda')
image_processor = vision_tower.image_processor

import os
import requests
from PIL import Image
from io import BytesIO
from llava.conversation import conv_templates, SeparatorStyle
from llava.utils import disable_torch_init
from llava.constants import IMAGE_TOKEN_INDEX, DEFAULT_IMAGE_TOKEN, DEFAULT_IM_START_TOKEN, DEFAULT_IM_END_TOKEN
from llava.mm_utils import tokenizer_image_token, get_model_name_from_path, KeywordsStoppingCriteria
from transformers import TextStreamer

def caption_image(image_file, prompt):
    if image_file.startswith('http') or image_file.startswith('https'):
        response = requests.get(image_file)
        image = Image.open(BytesIO(response.content)).convert('RGB')
    else:
        image = Image.open(image_file).convert('RGB')
    disable_torch_init()
    conv_mode = "llava_v0"
    conv = conv_templates[conv_mode].copy()
    roles = conv.roles
    image_tensor = image_processor.preprocess(image, return_tensors='pt')['pixel_values'].half().cuda()
    inp = f"{roles[0]}: {prompt}"
    inp = DEFAULT_IM_START_TOKEN + DEFAULT_IMAGE_TOKEN + DEFAULT_IM_END_TOKEN + '\n' + inp
    conv.append_message(conv.roles[0], inp)
    conv.append_message(conv.roles[1], None)
    raw_prompt = conv.get_prompt()
    input_ids = tokenizer_image_token(raw_prompt, tokenizer, IMAGE_TOKEN_INDEX, return_tensors='pt').unsqueeze(0).cuda()
    stop_str = conv.sep if conv.sep_style != SeparatorStyle.TWO else conv.sep2
    keywords = [stop_str]
    stopping_criteria = KeywordsStoppingCriteria(keywords, tokenizer, input_ids)
    with torch.inference_mode():
      output_ids = model.generate(input_ids, images=image_tensor, do_sample=True, temperature=0.2,
                                  max_new_tokens=1024, use_cache=True, stopping_criteria=[stopping_criteria])
    outputs = tokenizer.decode(output_ids[0, input_ids.shape[1]:]).strip()
    conv.messages[-1][-1] = outputs
    output = outputs.rsplit('</s>', 1)[0]
    return image, output

/content
Cloning into 'LLaVA'...
remote: Enumerating objects: 1529, done.
remote: Counting objects: 100% (380/380), done.
remote: Compressing objects: 100% (56/56), done.
remote: Total 1529 (delta 345), reused 324 (delta 324), pack-reused 1149
Receiving objects: 100% (1529/1529), 13.21 MiB | 8.64 MiB/s, done.
Resolving deltas: 100% (919/919), done.
/content/LLaVA
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 79.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.9/92.9 kB 13.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.7/302.7 kB 36.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 19.5 MB/s eta 0:00:00
     ━

config.json:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/33.7k [00:00<?, ?B/s]

pytorch_model-00001-of-00009.bin:   0%|          | 0.00/2.97G [00:00<?, ?B/s]

pytorch_model-00002-of-00009.bin:   0%|          | 0.00/2.93G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/749 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.71G [00:00<?, ?B/s]

## Set encoding




In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

## Dataset Split - Train & Test

In [ ]:
import shutil

# Source and destination paths
images = '/content/drive/MyDrive/Multimodal_Setiment_Analysis/Dataset/archive.zip'
destination_folder = '/content/'

# Copy the file from Drive to Colab
shutil.copy(images, destination_folder)

# Source and destination paths
labels = '/content/drive/MyDrive/Multimodal_Setiment_Analysis/Dataset/memotion_dataset_7k/labels.csv'
destination_folder = '/content/'

# Copy the file from Drive to Colab
shutil.copy(labels, destination_folder)



'/content/labels.csv'

In [ ]:
!unzip -j "/content/archive.zip" -d "/content/images/"

Streaming output truncated to the last 5000 lines.
  inflating: /content/images/image_2798.png  
  inflating: /content/images/image_2799.png  
  inflating: /content/images/image_28.jpg  
  inflating: /content/images/image_280.jpg  
  inflating: /content/images/image_2800.png  
  inflating: /content/images/image_2801.png  
  inflating: /content/images/image_2802.png  
  inflating: /content/images/image_2803.png  
  inflating: /content/images/image_2804.png  
  inflating: /content/images/image_2805.jpg  
  inflating: /content/images/image_2806.jpg  
  inflating: /content/images/image_2807.jpg  
  inflating: /content/images/image_2808.jpg  
  inflating: /content/images/image_2809.png  
  inflating: /content/images/image_281.png  
  inflating: /content/images/image_2810.jpg  
  inflating: /content/images/image_2811.jpg  
  inflating: /content/images/image_2812.jpg  
  inflating: /content/images/image_2813.jpg  
  inflating: /content/images/image_2814.jpg  
  inflating: /content/images/imag

In [ ]:
#check number of files in /content/data/images

import os

# Directory path
directory_path = '/content/images'

# List all files in the directory
files_list = os.listdir(directory_path)

# Count the number of files
num_files = len(files_list)

print(f"The number of files in the directory '{directory_path}' is: {num_files}")


The number of files in the directory '/content/images' is: 6998


### Training Split

In [ ]:
import pandas as pd
import os
import shutil
import random

# Load the dataset
dataset_path = '/content/labels.csv'
df = pd.read_csv(dataset_path)

# Print the number of images in the dataset
print(f"Total Number of images: {len(df)}")

print("#" * 40)

# Print the number of images for each sentiment class
print("Total Number of images per sentiment class:")

print("#" * 40)

print(df['overall_sentiment'].value_counts())

print("#" * 40)

# Define the desired ratios for each sentiment class
ratios = {'positive': 3127, 'neutral': 2201, 'very_positive': 1033, 'negative': 480, 'very_negative': 151}

print("#" * 40)

print("Number of images for training divided by sentiment class:")

# Create the "Training" folder if it doesn't exist
training_folder = '/content/train'
os.makedirs(training_folder, exist_ok=True)

# Shuffle the dataframe
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

# Initialize counters for each sentiment class
counters = {sentiment: 0 for sentiment in ratios.keys()}

# Iterate through the dataframe and copy images to the "Training" folder
for index, row in df.iterrows():
    image_name = row['image_name']
    sentiment = row['overall_sentiment']

    # Check if the number of images for the current sentiment class has reached the desired ratio
    if counters[sentiment] < ratios[sentiment]:
        # Copy the image to the "Training" folder
        src_path = os.path.join('/content/images', image_name)  # Replace with the actual path to your images
        dest_path = os.path.join(training_folder, image_name)
        shutil.copy(src_path, dest_path)

        # Increment the counter for the current sentiment class
        counters[sentiment] += 1

    # Check if the total number of copied images has reached 700
    if sum(counters.values()) >= 700:
        break

# Print the counts for each sentiment class after copying
for sentiment, count in counters.items():
    print(f"Number of '{sentiment}' images: {count}")

print("#" * 40)

# Calculate and print the actual ratios
actual_ratios = {sentiment: count / sum(counters.values()) for sentiment, count in counters.items()}
print("\nActual Ratios:")
print(actual_ratios)

# copy the df dataframe to a new dataframe train_df
train_df = df.copy()

# remove the rows that were not copied to the "Training" folder from the train_df dataframe
train_df = train_df.iloc[:sum(counters.values())]

# remove all columns except "image_name", "overall_sentiment" and "text_ocr"
train_df = train_df.drop(['text_corrected', 'humour', 'sarcasm', 'offensive', 'motivational'], axis=1)


# save the remaining rows to a new CSV file
train_df.to_csv('/content/train.csv', index=False)

print("#" * 40)

print("\nSaved the train.csv file successfully!")

Total Number of images: 6992
########################################
Total Number of images per sentiment class:
########################################
positive         3127
neutral          2201
very_positive    1033
negative          480
very_negative     151
Name: overall_sentiment, dtype: int64
########################################
########################################
Number of images for training divided by sentiment class:
Number of 'positive' images: 299
Number of 'neutral' images: 199
Number of 'very_positive' images: 119
Number of 'negative' images: 67
Number of 'very_negative' images: 16
########################################

Actual Ratios:
{'positive': 0.42714285714285716, 'neutral': 0.2842857142857143, 'very_positive': 0.17, 'negative': 0.09571428571428571, 'very_negative': 0.022857142857142857}
########################################

Saved the train.csv file successfully!


### Testing Split

In [ ]:
import pandas as pd
import os
import shutil
import random

# Load the dataset
dataset_path = '/content/labels.csv'
df = pd.read_csv(dataset_path)

# Load the training dataset
train_df = pd.read_csv('/content/train.csv')

# Exclude images used in the training dataset
df = df[~df['image_name'].isin(train_df['image_name'])]

# Print the number of images in the dataset
print(f"Total Number of images: {len(df)}")

print("#" * 40)

# Print the number of images for each sentiment class
print("Total Number of images per sentiment class:")

print("#" * 40)

print(df['overall_sentiment'].value_counts())

print("#" * 40)

# Define the desired ratios for each sentiment class
ratios = {'positive': 3127, 'neutral': 2201, 'very_positive': 1033, 'negative': 480, 'very_negative': 151}

print("#" * 40)

print("Number of images for testing divided by sentiment class:")

# Create the "Testing" folder if it doesn't exist
testing_folder = '/content/test'
os.makedirs(testing_folder, exist_ok=True)

# Shuffle the dataframe
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

# Initialize counters for each sentiment class
counters = {sentiment: 0 for sentiment in ratios.keys()}

# Iterate through the dataframe and copy images to the "Testing" folder
for index, row in df.iterrows():
    image_name = row['image_name']
    sentiment = row['overall_sentiment']

    # Check if the number of images for the current sentiment class has reached the desired ratio
    if counters[sentiment] < ratios[sentiment]:
        # Copy the image to the "Testing" folder
        src_path = os.path.join('/content/images', image_name)  # Replace with the actual path to your images
        dest_path = os.path.join(testing_folder, image_name)
        shutil.copy(src_path, dest_path)

        # Increment the counter for the current sentiment class
        counters[sentiment] += 1

    # Check if the total number of copied images has reached 300
    if sum(counters.values()) >= 300:
        break

# Print the counts for each sentiment class after copying
for sentiment, count in counters.items():
    print(f"Number of '{sentiment}' images: {count}")

print("#" * 40)

# Calculate and print the actual ratios
actual_ratios = {sentiment: count / sum(counters.values()) for sentiment, count in counters.items()}
print("\nActual Ratios:")
print(actual_ratios)

# copy the df dataframe to a new dataframe test_df
test_df = df.copy()

# remove the rows that were not copied to the "Testing" folder from the test_df dataframe
test_df = test_df.iloc[:sum(counters.values())]

# remove all columns except "image_name", "overall_sentiment" and "text_ocr"
test_df = test_df.drop(['text_corrected', 'humour', 'sarcasm', 'offensive', 'motivational'], axis=1)


# save the remaining rows to a new CSV file
test_df.to_csv('/content/test.csv', index=False)

print("#" * 40)

print("\nSaved the test.csv file successfully!")

Total Number of images: 6292
########################################
Total Number of images per sentiment class:
########################################
positive         2828
neutral          2002
very_positive     914
negative          413
very_negative     135
Name: overall_sentiment, dtype: int64
########################################
########################################
Number of images for testing divided by sentiment class:
Number of 'positive' images: 127
Number of 'neutral' images: 93
Number of 'very_positive' images: 49
Number of 'negative' images: 24
Number of 'very_negative' images: 7
########################################

Actual Ratios:
{'positive': 0.42333333333333334, 'neutral': 0.31, 'very_positive': 0.16333333333333333, 'negative': 0.08, 'very_negative': 0.023333333333333334}
########################################

Saved the test.csv file successfully!


# Training Pipeline

## Batch processing & LLM Response

In [ ]:
!unzip -j "/content/remaining_final.zip" -d "/content/train/"

Archive:  /content/remaining_final.zip
  inflating: /content/train/image_6537.jpg  
  inflating: /content/train/._image_6537.jpg  
  inflating: /content/train/image_3696.jpeg  
  inflating: /content/train/._image_3696.jpeg  
  inflating: /content/train/image_385.jpg  
  inflating: /content/train/._image_385.jpg  
  inflating: /content/train/image_1933.jpg  
  inflating: /content/train/._image_1933.jpg  
  inflating: /content/train/image_6669.jpg  
  inflating: /content/train/._image_6669.jpg  
  inflating: /content/train/image_568.jpg  
  inflating: /content/train/._image_568.jpg  
  inflating: /content/train/image_3260.jpg  
  inflating: /content/train/._image_3260.jpg  
  inflating: /content/train/image_4269.jpg  
  inflating: /content/train/._image_4269.jpg  
  inflating: /content/train/image_2618.jpg  
  inflating: /content/train/._image_2618.jpg  
  inflating: /content/train/image_6126.jpg  
  inflating: /content/train/._image_6126.jpg  
  inflating: /content/train/image_5407.jpg 

In [ ]:
import os
import pandas as pd

temp_dir = '/content/drive/MyDrive/Multimodal_Setiment_Analysis/Dataset/Final_Memotion_Dataset - Training (2000, 700) and Training (300)/Test/Humor_Data_300/total'

# Create an empty df to store the results.
results_df = pd.DataFrame(columns=['image_name', 'Image_Representation'])

# Iterate through the extracted images.
for i, image_file in enumerate(os.listdir(temp_dir)):
    image_path = os.path.join(temp_dir, image_file)

    # Extract file name without extension
    image_name = os.path.splitext(image_file)[0]

    prompt = "Please provide a detailed description of the image, paying close attention to its elements, colors, and overall composition. This information will be utilized for humour classification. The humour classes include: 1) Funny 2) Hilarious 3) Not Funny 4) Very Funny  Your thoughtful description will contribute to accurately classifying the humour associated with the image. Thank you!"

    image, label = caption_image(image_path, prompt)

    # Append the results to the DataFrame.
    results_df = results_df.append({'image_name': image_name, 'Image_Representation': label}, ignore_index=True)

output_csv_file = '/content/output_train_labels_humour_300.csv'
results_df.to_csv(output_csv_file, index=False)

print(f'Labels saved to {output_csv_file}')


<ipython-input-6-ebace6a0c60f>:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'image_name': image_name, 'Image_Representation': label}, ignore_index=True)
<ipython-input-6-ebace6a0c60f>:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'image_name': image_name, 'Image_Representation': label}, ignore_index=True)
<ipython-input-6-ebace6a0c60f>:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'image_name': image_name, 'Image_Representation': label}, ignore_index=True)
<ipython-input-6-ebace6a0c60f>:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat inst

Labels saved to /content/output_train_labels_humour_300.csv


<ipython-input-6-ebace6a0c60f>:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'image_name': image_name, 'Image_Representation': label}, ignore_index=True)


## Final Cleaning and Processing

In [ ]:
import os
import pandas as pd

images_folder = '/content/train'  # Update to your images folder path

labels_csv = '/content/labels.csv'  # Update to your labels file path

output_csv = '/content/output_train_labels_humour.csv'  # Update to your output file path

In [ ]:
# Load the labels.csv and output_labels.csv as DataFrames.

labels_df = pd.read_csv(labels_csv)

output_df = pd.read_csv(output_csv)

In [ ]:
labels_df

In [ ]:
output_df

In [ ]:
labels_df = labels_df.drop(['Unnamed: 0'], axis=1)

In [ ]:
labels_df['image_name'] = labels_df['image_name'].str.replace(r'\..*$', '', regex=True)

In [ ]:
labels_df.head()

,image_name,text_ocr,overall_sentiment
0,image_2818,WHAT IF THE MORPHEUS MEME IS REALLY JUST A BAD...,positive
1,image_2238,GIRLS ADJUST THEIR BRAS WHENEVER THEY WANT AND...,negative
2,image_2941,SWIMMING WITH THE JELLIES FRIGHTEOUS,very_positive
3,image_2952,FINDING EMO2 no dad,negative
4,image_2370,YOU'RE AFRAID OF REJECTION? WHAT'S THAT?,neutral


In [ ]:
output_df.head()

,image_name,Image_Representation
0,image_3185,"The image features a cartoon character, possib..."
1,image_3292,The image features a man wearing a suit and ti...
2,image_4791,The image features a man with a beard and a go...
3,image_4165,The image features a man with a ring on his fi...
4,image_1260,"The image features a woman with blonde hair, w..."


In [ ]:
merged_df = pd.merge(labels_df, output_df[['image_name', 'Image_Representation']], on='image_name', how='inner')

In [ ]:
merged_df.head()

,image_name,text_ocr,overall_sentiment,Image_Representation
0,image_2818,WHAT IF THE MORPHEUS MEME IS REALLY JUST A BAD...,positive,The image features a man with a surprised or s...
1,image_2238,GIRLS ADJUST THEIR BRAS WHENEVER THEY WANT AND...,negative,The image features a man with a white shirt an...
2,image_2941,SWIMMING WITH THE JELLIES FRIGHTEOUS,very_positive,"The image features a smiling turtle, possibly ..."
3,image_2952,FINDING EMO2 no dad,negative,"The image features a black and white fish, pos..."
4,image_2370,YOU'RE AFRAID OF REJECTION? WHAT'S THAT?,neutral,The image features a man with a beard and must...


In [ ]:
merged_df.to_csv('/content/Humor_Final_Dataset_2000.csv')

# Testing Pipeline

## Batch processing & LLM Response

In [ ]:
import os
import pandas as pd

temp_dir = '/content/test'

# Create an empty df to store the results.
results_df = pd.DataFrame(columns=['image_name', 'Image_Representation'])

# Iterate through the extracted images.
for i, image_file in enumerate(os.listdir(temp_dir)):
    image_path = os.path.join(temp_dir, image_file)

    # Extract file name without extension
    image_name = os.path.splitext(image_file)[0]

    prompt = 'Describe the image carefully. This description would be further needed for sentiment classification. The classes for sentiment analysis are 1) negative 2) neutral 3) positive 4) very_negative 5) very_positive'  # Customize this as needed
    image, label = caption_image(image_path, prompt)

    # Append the results to the DataFrame.
    results_df = results_df.append({'image_name': image_name, 'Image_Representation': label}, ignore_index=True)

output_csv_file = '/content/output_test_labels.csv'
results_df.to_csv(output_csv_file, index=False)

print(f'Labels saved to {output_csv_file}')


## Final Cleaning and Processing

In [ ]:
import os
import pandas as pd

images_folder = '/content/test'  # Update to your images folder path

labels_csv = '/content/test.csv'  # Update to your labels file path

output_csv = '/content/output_test_labels.csv'  # Update to your output file path

In [ ]:
# Load the labels.csv and output_labels.csv as DataFrames.

labels_df = pd.read_csv(labels_csv)

output_df = pd.read_csv(output_csv)

In [ ]:
labels_df = labels_df.drop(['Unnamed: 0'], axis=1)

In [ ]:
labels_df['image_name'] = labels_df['image_name'].str.replace(r'\..*$', '', regex=True)

In [ ]:
labels_df.shape

NameError: ignored

In [ ]:
output_df.head()

,image_name,Image_Representation
0,image_4533,The image features three young men posing for ...
1,image_478,The image features a man and a woman standing ...
2,image_4579,"The image features a man sitting in a chair, w..."
3,image_5610,The image features a man wearing a tuxedo and ...
4,image_5908,"The image features a man with a bald head, who..."


In [ ]:
merged_df = pd.merge(labels_df, output_df[['image_name', 'Image_Representation']], on='image_name', how='inner')

In [ ]:
merged_df.head()

,image_name,text_ocr,overall_sentiment,Image_Representation
0,image_2057,Guy Goes To Mexico To Kill Himself,positive,The image features a man standing in front of ...
1,image_6431,They have gun control in Cuba. They have unive...,neutral,The image features a man wearing a suit and ti...
2,image_4541,I SEE YOU GUYS ARE REVIVING OLD MEMES,very_positive,The image features a young man wearing a grey ...
3,image_1386,When you were a dead meme but now you're actua...,neutral,The image features a man wearing a purple suit...
4,image_6166,EVERYTHING IS GOING ACCORDING TO PLAN quickmem...,positive,The image features a large gray cat lying on a...


In [ ]:
merged_df.to_csv('/content/dataset_test_final.csv')